<a href="https://colab.research.google.com/github/anubhavgupta1/Dive-Into-Deep-Learning/blob/main/Recurrent%20Neural%20Networks/Text%20Pre%20Processing(%20Part2%20)/mxnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("drive/Udacity/RNN/")
!ls

sample_data
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0

In [2]:
!pip install d2l==0.16.1
!pip install -U mxnet-cu101==1.7.0

     |████████████████████████████████| 81kB 4.5MB/s 
     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x66334000 @  0x7fb17dfa1615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 20kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [3]:
%matplotlib inline
from d2l import mxnet as d2l
from mxnet import image, npx, np

npx.set_np()

In [4]:
import sys
sys.path.insert(0,'..')
from mxnet import nd
import random

### Reading Data

In [5]:
with open('timemachine.txt','r') as f:
  lines = f.readlines()
  raw_dataset = ' '.join(' '.join(lines).lower().split())

In [6]:
print("total characters in dataset : ",len(raw_dataset))
print("Sample dataset : ",raw_dataset[:100])

total characters in dataset :  178605
Sample dataset :  the time machine, by h. g. wells [1898] i the time traveller (for so it will be convenient to speak 


### Character Index

In [7]:
idx_to_char = list(set(raw_dataset))
print("total unique characters in dataset : ",len(idx_to_char))
print(idx_to_char)

total unique characters in dataset :  44
['b', '8', '9', '!', 'q', 'h', '(', '_', 'e', '"', 'm', 'g', 'j', 'l', 'd', 'y', ' ', 'p', '1', 'f', 'i', ')', 'u', 'v', '.', 'w', 'k', "'", 'o', '?', 'x', ';', 'n', 'c', 'a', ',', 't', 'r', '[', ']', ':', 's', '-', 'z']


In [8]:
char_to_idx = dict([ (char, i) for i, char in enumerate(idx_to_char)])
print(char_to_idx)

{'b': 0, '8': 1, '9': 2, '!': 3, 'q': 4, 'h': 5, '(': 6, '_': 7, 'e': 8, '"': 9, 'm': 10, 'g': 11, 'j': 12, 'l': 13, 'd': 14, 'y': 15, ' ': 16, 'p': 17, '1': 18, 'f': 19, 'i': 20, ')': 21, 'u': 22, 'v': 23, '.': 24, 'w': 25, 'k': 26, "'": 27, 'o': 28, '?': 29, 'x': 30, ';': 31, 'n': 32, 'c': 33, 'a': 34, ',': 35, 't': 36, 'r': 37, '[': 38, ']': 39, ':': 40, 's': 41, '-': 42, 'z': 43}


In [9]:
vocab_size = len(char_to_idx)
print("Vocab size is : ", vocab_size)

Vocab size is :  44


### Index to Text

In [10]:
corpus_indices = [char_to_idx[char] for char in raw_dataset]
print(corpus_indices[:10])
sample = corpus_indices[:100]

[36, 5, 8, 16, 36, 20, 10, 8, 16, 10]


In [11]:
print("Indices in sample : ", sample)
print("Sample text : ",''.join([ idx_to_char[idx] for idx in sample]))

Indices in sample :  [36, 5, 8, 16, 36, 20, 10, 8, 16, 10, 34, 33, 5, 20, 32, 8, 35, 16, 0, 15, 16, 5, 24, 16, 11, 24, 16, 25, 8, 13, 13, 41, 16, 38, 18, 1, 2, 1, 39, 16, 20, 16, 36, 5, 8, 16, 36, 20, 10, 8, 16, 36, 37, 34, 23, 8, 13, 13, 8, 37, 16, 6, 19, 28, 37, 16, 41, 28, 16, 20, 36, 16, 25, 20, 13, 13, 16, 0, 8, 16, 33, 28, 32, 23, 8, 32, 20, 8, 32, 36, 16, 36, 28, 16, 41, 17, 8, 34, 26, 16]
Sample text :  the time machine, by h. g. wells [1898] i the time traveller (for so it will be convenient to speak 


### Random Sampling

In [12]:
def seq_data_iter_random(corpus, batch_size, num_steps, verbose = False):  #@save
    """Generate a minibatch of subsequences using random sampling."""
    
    # Start with a random offset (inclusive of `num_steps - 1`) to partition a
    # sequence
    
    offset = random.randint(0, num_steps - 1)
    corpus = corpus[offset:]    
    # Subtract 1 since we need to account for labels
    num_subseqs = (len(corpus) - 1) // num_steps
    if verbose :
        print("Offset is : {OS} and Corpus length is : {cps}\n".format(OS = offset, cps = len(corpus)))
        print("Steps : ", num_steps)
        print("\nNumber of subsequences are : ",num_subseqs,"\n")
    # The starting indices for subsequences of length `num_steps`
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # In random sampling, the subsequences from two adjacent random
    # minibatches during iteration are not necessarily adjacent on the
    # original sequence
    random.shuffle(initial_indices)

    def data(pos):
        # Return a sequence of length `num_steps` starting from `pos`
        return corpus[pos: pos + num_steps]

    num_batches = num_subseqs // batch_size
    
    if verbose :
        print("Number of batches : ",num_batches," and batch size is : ", batch_size,"\n")

    for i in range(0, batch_size * num_batches, batch_size):
        # Here, `initial_indices` contains randomized starting indices for
        # subsequences
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        
        if verbose : 
            print(i," to " ,i + batch_size-1," initial indices per batch : ",initial_indices_per_batch,"\n")
        
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield np.array(X), np.array(Y)

In [13]:
my_seq = list(range(35))
for X, Y in seq_data_iter_random(my_seq, batch_size=2, num_steps=5, verbose = True):
    print('\nX:\n ', X, '\n\nY:\n', Y,"\n\n")

Offset is : 1 and Corpus length is : 34

Steps :  5

Number of subsequences are :  6 

Number of batches :  3  and batch size is :  2 

0  to  1  initial indices per batch :  [15, 10] 


X:
  [[16. 17. 18. 19. 20.]
 [11. 12. 13. 14. 15.]] 

Y:
 [[17. 18. 19. 20. 21.]
 [12. 13. 14. 15. 16.]] 


2  to  3  initial indices per batch :  [0, 25] 


X:
  [[ 1.  2.  3.  4.  5.]
 [26. 27. 28. 29. 30.]] 

Y:
 [[ 2.  3.  4.  5.  6.]
 [27. 28. 29. 30. 31.]] 


4  to  5  initial indices per batch :  [20, 5] 


X:
  [[21. 22. 23. 24. 25.]
 [ 6.  7.  8.  9. 10.]] 

Y:
 [[22. 23. 24. 25. 26.]
 [ 7.  8.  9. 10. 11.]] 




In [14]:
def seq_data_iter_sequential(corpus, batch_size, num_steps, verbose = False):  #@save
    """Generate a minibatch of subsequences using sequential partitioning."""
    # Start with a random offset to partition a sequence
    offset = random.randint(0, num_steps)
    num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size

    if verbose :
          print("Offset is : ", offset," Number of tokens are : ", num_tokens)

    Xs = np.array(corpus[offset: offset + num_tokens])
    Ys = np.array(corpus[offset + 1: offset + 1 + num_tokens])

    if verbose :
        print("\nX : ", Xs.shape," Y : ", Ys.shape)

    Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1)
    num_batches = Xs.shape[1] // num_steps
    
    if verbose :
        print("\nX : ", Xs.shape," Y : ", Ys.shape)
        print("\nNumber of batches : ", num_batches)
    
    for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps]
        Y = Ys[:, i: i + num_steps]
        yield X, Y

In [15]:
my_seq = list(range(35))
for X, Y in seq_data_iter_sequential(my_seq, batch_size=2, num_steps=5, verbose = True):
    print('\nX:\n ', X, '\n\nY:\n', Y,"\n\n")

Offset is :  1  Number of tokens are :  32

X :  (32,)  Y :  (32,)

X :  (2, 16)  Y :  (2, 16)

Number of batches :  3

X:
  [[ 1.  2.  3.  4.  5.]
 [17. 18. 19. 20. 21.]] 

Y:
 [[ 2.  3.  4.  5.  6.]
 [18. 19. 20. 21. 22.]] 



X:
  [[ 6.  7.  8.  9. 10.]
 [22. 23. 24. 25. 26.]] 

Y:
 [[ 7.  8.  9. 10. 11.]
 [23. 24. 25. 26. 27.]] 



X:
  [[11. 12. 13. 14. 15.]
 [27. 28. 29. 30. 31.]] 

Y:
 [[12. 13. 14. 15. 16.]
 [28. 29. 30. 31. 32.]] 


